In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/transactions-fraud-datasets/mcc_codes.json
/kaggle/input/transactions-fraud-datasets/users_data.csv
/kaggle/input/transactions-fraud-datasets/transactions_data.csv
/kaggle/input/transactions-fraud-datasets/cards_data.csv
/kaggle/input/transactions-fraud-datasets/train_fraud_labels.json


In [2]:
import numpy as np
import pandas as pd 
import json
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots                                                         
import plotly.io as pio
pio.renderers.default="iframe_connected"
from itertools import islice
import warnings
warnings.filterwarnings("ignore")

##  Transaction Data

In [3]:
txn_df = pd.read_csv('/kaggle/input/transactions-fraud-datasets/transactions_data.csv')
txn_df

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13305910,23761868,2019-10-31 23:56:00,1718,2379,$1.11,Chip Transaction,86438,West Covina,CA,91792.0,5499,NaN
13305911,23761869,2019-10-31 23:56:00,1766,2066,$12.80,Online Transaction,39261,ONLINE,NaN,NaN,5815,NaN
13305912,23761870,2019-10-31 23:57:00,199,1031,$40.44,Swipe Transaction,2925,Allen,TX,75002.0,4900,NaN
13305913,23761873,2019-10-31 23:58:00,1986,5443,$4.00,Chip Transaction,46284,Daly City,CA,94014.0,5411,NaN


In [4]:
txn_df.describe()

,id,client_id,card_id,merchant_id,zip,mcc
count,1.330592e+07,1.330592e+07,1.330592e+07,1.330592e+07,1.165321e+07,1.330592e+07
mean,1.558402e+07,1.026812e+03,3.475268e+03,4.772376e+04,5.132782e+04,5.565440e+03
std,4.704499e+06,5.816386e+02,1.674356e+03,2.581534e+04,2.940423e+04,8.757002e+02
min,7.475327e+06,0.000000e+00,0.000000e+00,1.000000e+00,1.001000e+03,1.711000e+03
25%,1.150604e+07,5.190000e+02,2.413000e+03,2.588700e+04,2.860200e+04,5.300000e+03
50%,1.557087e+07,1.070000e+03,3.584000e+03,4.592600e+04,4.767000e+04,5.499000e+03
75%,1.965361e+07,1.531000e+03,4.901000e+03,6.757000e+04,7.790100e+04,5.812000e+03
max,2.376187e+07,1.998000e+03,6.144000e+03,1.003420e+05,9.992800e+04,9.402000e+03


In [5]:
txn_df.dtypes

id                  int64
date               object
client_id           int64
card_id             int64
amount             object
use_chip           object
merchant_id         int64
merchant_city      object
merchant_state     object
zip               float64
mcc                 int64
errors             object
dtype: object

In [6]:
txn_df.date = pd.to_datetime(txn_df.date) # convert date to datetime
txn_df.amount = txn_df.amount.str.replace('$', '').astype(float)
txn_df.dtypes

id                         int64
date              datetime64[ns]
client_id                  int64
card_id                    int64
amount                   float64
use_chip                  object
merchant_id                int64
merchant_city             object
merchant_state            object
zip                      float64
mcc                        int64
errors                    object
dtype: object

In [7]:
txn_df.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN


In [8]:
txn_df.isna().sum()

id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id              0
merchant_city            0
merchant_state     1563700
zip                1652706
mcc                      0
errors            13094522
dtype: int64

###   Handle NULL `merchant_state`

In [9]:
nan_state = txn_df[txn_df.merchant_state.isna()]

In [10]:
print(nan_state.shape)
nan_state.head()

(1563700, 12)


,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
7,7475335,2010-01-01 00:14:00,1684,2140,26.46,Online Transaction,39021,ONLINE,NaN,NaN,4784,NaN
8,7475336,2010-01-01 00:21:00,335,5131,261.58,Online Transaction,50292,ONLINE,NaN,NaN,7801,NaN
18,7475346,2010-01-01 00:34:00,394,4717,26.04,Online Transaction,39021,ONLINE,NaN,NaN,4784,NaN
24,7475353,2010-01-01 00:43:00,301,3742,10.17,Online Transaction,39021,ONLINE,NaN,NaN,4784,NaN
26,7475356,2010-01-01 00:45:00,566,3439,16.86,Online Transaction,16798,ONLINE,NaN,NaN,4121,NaN


In [11]:
nan_state.merchant_city.unique()

array(['ONLINE'], dtype=object)

In [12]:
nan_state.use_chip.unique()

array(['Online Transaction', 'Chip Transaction'], dtype=object)

In [13]:
txn_df.merchant_state.unique()

array(['ND', 'IA', 'CA', 'IN', 'MD', 'NY', nan, 'TX', 'HI', 'PA', 'WI',
       'GA', 'AL', 'CT', 'WA', 'MA', 'CO', 'NJ', 'OK', 'MT', 'FL', 'AZ',
       'KY', 'LA', 'IL', 'OH', 'MO', 'MI', 'KS', 'NC', 'AR', 'TN', 'NM',
       'SC', 'MN', 'NV', 'OR', 'VA', 'SD', 'WV', 'ME', 'MS', 'RI', 'NH',
       'DE', 'VT', 'Mexico', 'ID', 'NE', 'DC', 'UT', 'Vatican City', 'WY',
       'Dominican Republic', 'Canada', 'AK', 'Costa Rica', 'Germany',
       'China', 'United Kingdom', 'Estonia', 'Tuvalu', 'Taiwan',
       'United Arab Emirates', 'Lithuania', 'Netherlands', 'Japan',
       'Greece', 'Vietnam', 'Haiti', 'Ireland', 'Singapore', 'France',
       'South Africa', 'Thailand', 'Italy', 'Denmark', 'Jamaica', 'Benin',
       'Belgium', 'Sierra Leone', 'Indonesia', 'Colombia', 'Switzerland',
       'Portugal', 'New Zealand', 'Jordan', 'Guatemala', 'Hong Kong',
       'Finland', 'Mongolia', 'Saudi Arabia', 'Philippines', 'Norway',
       'Hungary', 'South Korea', 'Israel', 'India', 'Australia', 'Mona

In [14]:
txn_df.merchant_city.unique()

array(['Beulah', 'Bettendorf', 'Vista', ..., 'Chester Heights',
       'Shawboro', 'King Ferry'], dtype=object)

In [15]:
txn_df.merchant_state.fillna('ONLINE', inplace = True)
txn_df.isna().sum()

id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id              0
merchant_city            0
merchant_state           0
zip                1652706
mcc                      0
errors            13094522
dtype: int64

###   Handle NULL `zip`

In [16]:
nan_zip = txn_df[txn_df.zip.isna()]

In [17]:
nan_zip.merchant_city.unique()

array(['ONLINE', 'Puerto Vallarta', 'Vatican City', 'Guadalajara',
       'Santo Domingo', 'Montreal', 'Toronto', 'San Jose', 'Berlin',
       'Mexico City', 'Shanghai', 'Cancun', 'Edinburgh', 'Tallinn',
       'Funafuti', 'Tapei', 'Abu Dhabi', 'Beijing', 'Vilnius',
       'Amsterdam', 'Tokyo', 'Athens', 'Ho Chi Minh City',
       'Port au Prince', 'Dublin', 'Singapore', 'Paris', 'Johannesberg',
       'Cabo San Lucas', 'Bangkok', 'Rome', 'Copenhagen', 'Kingston',
       'Porto-Novo', 'London', 'Edmonton', 'Calgary', 'Brussels',
       'Freetown', 'Jakarta', 'Bogota', 'Geneva', 'Vancouver', 'Lisbon',
       'Wellington', 'Amman', 'Guatamala City', 'Hong Kong', 'Helsinki',
       'Ulan Bator', 'Riyadh', 'Manila', 'Oslo', 'Budapest', 'Seoul',
       'Jerusalem', 'Mumbai', 'Sydney', 'Monaco', 'Bucharest',
       'Saint Petersburg', 'Vienna', 'Majuro', 'Delhi', 'Barcelona',
       'Zurich', 'Oranjestad', 'Lima', 'Chisinau', 'Acapulco', 'Dhaka',
       'Madrid', 'Sao Paolo', 'Palikir', 'Tas

In [18]:
txn_df.loc[txn_df["merchant_city"] == "ONLINE", 'zip'] == 0 
txn_df.isna().sum()

id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id              0
merchant_city            0
merchant_state           0
zip                1652706
mcc                      0
errors            13094522
dtype: int64

In [19]:
# Lookup I got, just to fill in values. Can skip if you want
missing_zip = {
  "Puerto Vallarta": "48300",
  "Vatican City": "00120",
  "Guadalajara": "44100",
  "Santo Domingo": "10101",
  "Montreal": "H3A",
  "Toronto": "M5H",
  "San Jose": "10101",
  "Berlin": "10115",
  "Mexico City": "01000",
  "Shanghai": "200000",
  "Cancun": "77500",
  "Edinburgh": "EH1",
  "Tallinn": "10111",
  "Funafuti": "",
  "Tapei": "100",
  "Abu Dhabi": "00000",
  "Beijing": "100000",
  "Vilnius": "01100",
  "Amsterdam": "1011",
  "Tokyo": "100-0001",
  "Athens": "10552",
  "Ho Chi Minh City": "700000",
  "Port au Prince": "6110",
  "Dublin": "D01",
  "Singapore": "018989",
  "Paris": "75001",
  "Johannesberg": "",
  "Cabo San Lucas": "23450",
  "Bangkok": "10200",
  "Rome": "00184",
  "Copenhagen": "1050",
  "Kingston": "",
  "Porto-Novo": "",
  "London": "SW1A 1AA",
  "Edmonton": "T5J",
  "Calgary": "T2P",
  "Brussels": "1000",
  "Freetown": "",
  "Jakarta": "10110",
  "Bogota": "110111",
  "Geneva": "1201",
  "Vancouver": "V5K",
  "Lisbon": "1100-148",
  "Wellington": "6011",
  "Amman": "11118",
  "Guatamala City": "01001",
  "Hong Kong": "999077",
  "Helsinki": "00100",
  "Ulan Bator": "15160",
  "Riyadh": "12611",
  "Manila": "1000",
  "Oslo": "0150",
  "Budapest": "1051",
  "Seoul": "04524",
  "Jerusalem": "91000",
  "Mumbai": "400001",
  "Sydney": "2000",
  "Monaco": "98000",
  "Bucharest": "010011",
  "Saint Petersburg": "190000",
  "Vienna": "1010",
  "Majuro": "96960",
  "Delhi": "110001",
  "Barcelona": "08001",
  "Zurich": "8001",
  "Oranjestad": "",
  "Lima": "15001",
  "Chisinau": "MD-2001",
  "Acapulco": "39300",
  "Dhaka": "1000",
  "Madrid": "28001",
  "Sao Paulo": "01000-000",
  "Palikir": "96941",
  "Tashkent": "100000",
  "Santiago": "8320000",
  "Zagreb": "10000",
  "Kolkata": "700001",
  "Bangalore": "560001",
  "Rio de Janeiro": "20000-000",
  "Stockholm": "11120",
  "Bandar Seri Begawan": "BA1111",
  "Saint John's": "",
  "Andorra La Vella": "AD500",
  "Nairobi": "00100",
  "Prague": "110 00",
  "Nuku Alofa": "",
  "Asmara": "",
  "Honiara": "",
  "Islamabad": "44000",
  "Nassau": "",
  "Rabat": "10000",
  "Suva": "",
  "Muscat": "113",
  "Yaounde": "",
  "Skopje": "1000",
  "Tegucigalpa": "11101",
  "Abuja": "900001",
  "Belmopan": "",
  "Montevideo": "11000",
  "Istanbul": "34000",
  "Manama": "",
  "Moscow": "101000",
  "Kuala Lumpur": "50000",
  "Luxembourg": "L-1111",
  "Algiers": "16000",
  "Bridgetown": "BB11114",
  "Niamey": "",
  "Cairo": "11511",
  "Hanoi": "100000",
  "Accra": "00233",
  "Bratislava": "811 01",
  "Kiev": "01001",
  "Yaren District": "",
  "Buenos Aires": "C1000",
  "Juba": "",
  "Podgorica": "81000",
  "Colombo": "00100",
  "Lahore": "54000",
  "Reykjavik": "101",
  "Ljubljana": "1000",
  "Sanaa": "",
  "Riga": "LV-1050",
  "Doha": "00000",
  "Karachi": "74000",
  "Male": "20026",
  "Ouagadougou": "",
  "Harare": "",
  "Praia": "",
  "Malabo": "",
  "Port Vila": "",
  "Addis Ababa": "1000",
  "Mbabane": "",
  "Maputo": "1100",
  "Warsaw": "00-001",
  "Lusaka": "10101",
  "Kingstown": "",
  "Tblisi": "0105",
  "Baghdad": "10001",
  "Apia": "",
  "Monrovia": "1000",
  "Bamako": "",
  "Nicosia": "1010",
  "Quito": "170101",
  "Dakar": "",
  "Libreville": "",
  "Belgrade": "11000",
  "Tunis": "1000",
  "Pristina": "10000",
  "Bishek": "720001",
  "Victoria": "",
  "Sarajevo": "71000",
  "Panama City": "0819",
  "Paramaribo": "",
  "Yamoussoukro": "",
  "Yangon": "11181",
  "Tehran": "11369",
  "Beirut": "1107",
  "Valletta": "VLT 1117",
  "Tirana": "1001",
  "Conakry": "",
  "Port Moresby": "111",
  "Dili": "",
  "Caracas": "1010",
  "Khartoum": "11111",
  "Port of Spain": "",
  "Brazzaville": "",
  "Baku": "AZ1000",
  "Georgetown": "",
  "Sao Paolo": "01000-000"
}


In [20]:
txn_df["zip"] = txn_df["zip"].fillna(txn_df["merchant_city"].map(missing_zip))
txn_df.isna().sum()

id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id              0
merchant_city            0
merchant_state           0
zip                1563700
mcc                      0
errors            13094522
dtype: int64

###   Handle NULL `error`

In [21]:
txn_df[txn_df.errors.isna()].fillna('No Errors', inplace = True)
txn_df.isna().sum()

id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id              0
merchant_city            0
merchant_state           0
zip                1563700
mcc                      0
errors            13094522
dtype: int64

The above didn't work because of `chained indexing.`

Pandas does not guarantee whether: `df[condition]` returns a view or a copy — so inplace=True becomes unreliable.

In [22]:
txn_df['errors'].fillna('No Errors', inplace = True)
txn_df.isna().sum()

id                      0
date                    0
client_id               0
card_id                 0
amount                  0
use_chip                0
merchant_id             0
merchant_city           0
merchant_state          0
zip               1563700
mcc                     0
errors                  0
dtype: int64

In [23]:
txn_df.head(30)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,No Errors
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,No Errors
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,No Errors
3,7475331,2010-01-01 00:05:00,430,2860,200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,No Errors
4,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,No Errors
5,7475333,2010-01-01 00:07:00,1807,165,4.81,Swipe Transaction,20519,Bronx,NY,10464.0,5942,No Errors
6,7475334,2010-01-01 00:09:00,1556,2972,77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,No Errors
7,7475335,2010-01-01 00:14:00,1684,2140,26.46,Online Transaction,39021,ONLINE,ONLINE,NaN,4784,No Errors
8,7475336,2010-01-01 00:21:00,335,5131,261.58,Online Transaction,50292,ONLINE,ONLINE,NaN,7801,No Errors
9,7475337,2010-01-01 00:21:00,351,1112,10.74,Swipe Transaction,3864,Flushing,NY,11355.0,5813,No Errors


## Fraud Data

In [24]:
with open('/kaggle/input/transactions-fraud-datasets/train_fraud_labels.json', 'r') as file:
    fraud = json.load(file)

In [25]:
# list(fraud.items())[:3] # this gives me first 3 key-value paris, but everything is wrapped in target so we see full dict

list(fraud['target'].items())[:3] # this gives me first 3 key-value pairs

[('10649266', 'No'), ('23410063', 'No'), ('9316588', 'No')]

In [26]:
fraud_df = pd.DataFrame(fraud['target'].items(), columns=['transaction_id', 'is_fraud'])
fraud_df.head(3)

,transaction_id,is_fraud
0,10649266,No
1,23410063,No
2,9316588,No


In [27]:
print(txn_df.shape)
txn_df.dtypes

(13305915, 12)


id                         int64
date              datetime64[ns]
client_id                  int64
card_id                    int64
amount                   float64
use_chip                  object
merchant_id                int64
merchant_city             object
merchant_state            object
zip                       object
mcc                        int64
errors                    object
dtype: object

In [28]:
fraud_df.dtypes

transaction_id    object
is_fraud          object
dtype: object

In [29]:
fraud_df.transaction_id = fraud_df.transaction_id.astype('int64')
print(fraud_df.shape)
fraud_df.dtypes

(8914963, 2)


transaction_id     int64
is_fraud          object
dtype: object

In [30]:
txn = pd.merge(txn_df,fraud_df,left_on='id',right_on='transaction_id',how='inner')
txn.head(3)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,transaction_id,is_fraud
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,No Errors,7475327,No
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,No Errors,7475328,No
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,No Errors,7475329,No


In [31]:
txn.shape

(8914963, 14)

In [32]:
txn[txn.is_fraud == 'Yes'].shape

(13332, 14)

## EDA on Fraud Data